In [1]:
import numpy as np
import cv2
from kociemba import solve
import matplotlib.pyplot as plt
import serial

In [2]:
capFront = cv2.VideoCapture(5)
capBack = cv2.VideoCapture(3)
capFront.set(cv2.CAP_PROP_BRIGHTNESS,       150.0 )#0.50 default
capFront.set(cv2.CAP_PROP_CONTRAST,         100.0 )#0.15 default
capFront.set(cv2.CAP_PROP_SATURATION,       105.0 )#0.15 default
capBack.set(cv2.CAP_PROP_BRIGHTNESS,       150.0 )#0.50 default
capBack.set(cv2.CAP_PROP_CONTRAST,         100.0 )#0.15 default
capBack.set(cv2.CAP_PROP_SATURATION,       105.0 )#0.15 default
capFront.getBackendName()

'V4L2'

In [3]:
while True:
    ret1, frameFront = capFront.read()
    ret2, frameBack = capBack.read()
    if ret1:
        cv2.imshow('Front',frameFront)
        if(cv2.waitKey(1) & 0xFF == ord('q')):
            break
    if ret2:
        cv2.imshow('Back',frameBack)
        if(cv2.waitKey(1) & 0xFF == ord('q')):
            break
        
cv2.destroyAllWindows()

: 

In [4]:
import numpy as np
import cv2 as cv
import os
import sys

mylist = []

def print_coord(event,x,y,flags,param):
    if event == cv.EVENT_LBUTTONDOWN:
        mylist.append([x,y])
ret, img = capFront.read()
imgheight, imgwidth = img.shape[:2]
resizedImg = cv.resize(img,(int(imgwidth), int(imgheight)), interpolation = cv.INTER_AREA)
cv.namedWindow('Get Coordinates')
cv.setMouseCallback('Get Coordinates',print_coord)
cv.imshow('Get Coordinates',resizedImg)
cv.waitKey(0)
cv.destroyAllWindows()
print(mylist)
mat = np.array(mylist)

[[472, 416]]


In [18]:
mat = np.array([[376, 58],
                [455, 107],
                [515, 138],
                [382, 173], 
                [444, 239], 
                [528, 273], 
                [398, 308], 
                [473, 338], 
                [532, 376], 
                [119, 131], 
                [186, 106], 
                [281, 59], 
                [103, 269], 
                [207, 229], 
                [291, 171], 
                [100, 354], 
                [193, 334], 
                [295, 301], 
                [335, 388], 
                [421, 417], 
                [486, 436], 
                [219, 413], 
                [322, 421], 
                [388, 445], 
                [136, 431], 
                [220, 439], 
                [261, 448]])

retv, img2 = capFront.read()
for i in range(27):
    img3 = cv2.circle(img2,mat[i],10,(255,0,0),1)
cv2.imshow('kekw',img3)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [19]:
def guessColor( v, r ):
    colorCenters = (
    ( np.array([85.0, 75.0, 60.0]), np.array([150.0, 225.0, 255.0]),'B'),
    ( np.array([0.0, 70.0, 70.0]), np.array([20.0, 255.0, 255.0]),"O" ),
    ( np.array([20.5, 40.0, 35.0]),np.array([47.0, 255.0, 255.0]),  "Y" ),
    ( np.array([47.5, 35.0, 40.0]), np.array([84.0, 255.0, 255.0]),"G" ),
    (np.array([0.0, 0.0, 100.0]), np.array([180.0, 60.0, 255.0]) ,"W" ),
    ( np.array([160.0, 60.0,  80.0]), np.array([180.0, 255.0, 255.0]),"R" )
    )

    for min, max, code in colorCenters:
        if ((min[0] <= v[0] <= max[0]) and (min[1] <= v[1] <= max[1]) and (min[2] <= v[2] <= max[2])):
            if code == 'R':
                #print(r)
                if (r[0] < 100.0 or (r[0] < 140.0 and r[1] > 70.0 and  r[2] > 75.0) or (r[0] < 200 and r[1] > 120.0 and r[2] > 130.0)):
                    code = 'O'
            return code
    return "X"

In [20]:
ret, frameBack = capBack.read() 
if ret:
    rgb = cv2.cvtColor(frameBack, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(frameBack, cv2.COLOR_BGR2HSV)

    colorSamplesB = []
    codeB = []
    w = 8
    h = 8
    matBack = np.array([[496, 60], #Up
                    [459, 80],
                    [363, 114],
                    [431, 40],
                    [351, 80], #UpCenter
                    [257, 95],
                    [280, 37],
                    [250, 51],
                    [170, 80],
                    [131, 156], #Right
                    [221, 183],
                    [325, 219],
                    [138, 257],
                    [246, 279], #RightCenter
                    [332, 349],
                    [165, 386],
                    [237, 402],
                    [331, 448],
                    [419, 220], #Back
                    [498, 160],
                    [550, 117],
                    [421, 332],
                    [470, 255], #BackCenter
                    [542, 220],
                    [427, 434],
                    [482, 376],
                    [524, 355]])
    for lines in matBack:
        x1,y1 = lines
        blockHSV = hsv[(y1-h//2):(y1+h//2),(x1-w//2):(x1+w//2)]
        block_h = blockHSV[:,:, 0 ]
        block_s = blockHSV[:,:, 1 ]
        block_v = blockHSV[:,:, 2 ]
        blockRGB = rgb[(y1-h//2):(y1+h//2),(x1-w//2):(x1+w//2)]
        block_r = blockRGB[:,:, 0 ]
        block_g = blockRGB[:,:, 1 ]
        block_b = blockRGB[:,:, 2 ]
        #print(block_h)
        clrh = np.array((np.median(block_h),np.median( block_s ), np.median( block_v ))) 
        clrr = np.array((np.median(block_r),np.median( block_g ), np.median( block_b ))) 
        
        colorSamplesB.append(clrr)
        codeB.append(guessColor(clrh,clrr))
        #print(clr)
    for i in range(27):
        if codeB[i] == 'R':
            print(colorSamplesB[i])

        img3 = cv2.circle(frameBack,matBack[i],10,(255,0,0),1)
        font = cv2.FONT_HERSHEY_SIMPLEX
        img4 = cv2.putText(img3,codeB[i],matBack[i],font,0.8,(255,255,0),1,cv2.LINE_AA) 
    cv2.imshow('kekw',img4)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [21]:
ret, frameFront = capFront.read() 
hsv = cv2.cvtColor(frameFront, cv2.COLOR_BGR2HSV)
rgb = cv2.cvtColor(frameFront, cv2.COLOR_BGR2RGB)

colorSamplesF = []
codeF = []
w = 8
h = 8
matFront = np.array([[376, 58], #Front
                [450, 107],
                [505, 128],
                [382, 173], 
                [444, 239], #FrontCenter
                [528, 273], 
                [398, 308], 
                [473, 338], 
                [532, 376], 
                [119, 131], #Left
                [186, 106], 
                [281, 59], 
                [103, 269], 
                [207, 229], #LeftCenter
                [291, 171], 
                [100, 354], 
                [193, 334], 
                [295, 301], 
                [322, 398], #Down
                [411, 417], 
                [480, 425], 
                [224, 418], 
                [322, 421], #DownCenter
                [395, 442],
                [136, 431], 
                [225, 439], 
                [260, 453]])
for lines in matFront:
    x1,y1 = lines
    blockHSV = hsv[(y1-h//2):(y1+h//2),(x1-w//2):(x1+w//2)]
    block_h = blockHSV[:,:, 0 ]
    block_s = blockHSV[:,:, 1 ]
    block_v = blockHSV[:,:, 2 ]
    blockRGB = rgb[(y1-h//2):(y1+h//2),(x1-w//2):(x1+w//2)]
    block_r = blockRGB[:,:, 0 ]
    block_g = blockRGB[:,:, 1 ]
    block_b = blockRGB[:,:, 2 ]
    clrh = np.array((np.median(block_h),np.median( block_s ), np.median( block_v )))
    clrr = np.array((np.median(block_r),np.median( block_g ), np.median( block_b )))  
    codeF.append(guessColor(clrh, clrr))
    colorSamplesF.append(clrr)
    #print(clr)
codeF[26] = 'X'
for i in range(27):
    if codeF[i] == 'O':
        print(colorSamplesF[i])
    img3 = cv2.circle(frameFront,matFront[i],10,(255,0,0),1)
    font = cv2.FONT_HERSHEY_SIMPLEX
    img4 = cv2.putText(img3,codeF[i],matFront[i],font,0.8,(255,255,0),1,cv2.LINE_AA) 
cv2.imshow('kekw',img4)
cv2.waitKey(0)
cv2.destroyAllWindows()

[241.  176.   63.5]
[208.  105.5  24. ]
[206.  98.  26.]
[157.  75.  13.]
[219.5 107.   29. ]


In [251]:
colorCode = codeB[0:18] + codeF[0:9] + codeF[18:27] + codeF[9:18] + codeB[18:27]
len(colorCode)
print(colorCode)

['Y', 'O', 'O', 'R', 'G', 'B', 'B', 'W', 'B', 'O', 'Y', 'G', 'R', 'W', 'G', 'B', 'O', 'O', 'R', 'G', 'Y', 'Y', 'R', 'G', 'G', 'G', 'R', 'W', 'Y', 'Y', 'B', 'B', 'B', 'G', 'W', 'X', 'G', 'W', 'W', 'O', 'Y', 'R', 'O', 'R', 'R', 'W', 'W', 'R', 'O', 'O', 'Y', 'B', 'B', 'Y']


In [252]:
ser = serial.Serial()
ser.baudrate = 115200
ser.port = 'COM6'
ser.open()

In [253]:
colors = ['W','B','R','O','G','Y']
for color in colors:
    if colorCode.count(color) == 8:
        colorCode = list(map(lambda x: x.replace('X', color), colorCode))
    if colorCode.count(color) >= 10:
        ser.write('F U B D L R'.encode())

colorDict = {
    colorCode[4] : 'U',
    colorCode[13] : 'R',
    colorCode[22] : 'F',
    colorCode[31] : 'D',
    colorCode[40] : 'L',
    colorCode[49] : 'B'
}  





In [254]:
print(colorDict)

{'G': 'U', 'W': 'R', 'R': 'F', 'B': 'D', 'Y': 'L', 'O': 'B'}


In [255]:
cubestring = ""
colorstring = ""
colorstring = colorstring.join(colorCode)
cubestring = cubestring.join(map(colorDict.get,colorCode,colorCode))
for color in colorDict:
    print(color, '->', colorDict[color], cubestring.count(colorDict[color]))
print (colorstring, cubestring)

G -> U 9
W -> R 9
R -> F 9
B -> D 9
Y -> L 9
O -> B 9
YOORGBBWBOYGRWGBOORGYYRGGGRWYYBBBGWWGWWOYRORRWWROOYBBY LBBFUDDRDBLUFRUDBBFULLFUUUFRLLDDDURRURRBLFBFFRRFBBLDDL


In [256]:
solveString = solve(cubestring)
print(solveString)

U' R B2 D L' B U R' L B' D2 R2 U R2 D' F2 B2 U' L2 D'


In [257]:
ser.write(solveString.encode())


53

In [16]:
#ser.close()
capFront.release()
capBack.release()